In [1]:
import json
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from src.utils import construct_sparse_adj_mat
from pathlib import Path
from collections import Counter

In [2]:
fn = "/home/stef/quest_data/hiec/results/predictions/dev_clipzyme/sprhea_v3_folded_n_100_rcmcs_inner_fold_1_of_3/target_output.parquet"
df = pd.read_parquet(fn)
df.head()

,protein_idx,reaction_idx,pid,rid,y,logits
0,49,67,P30930,1707,1,0.730571
1,22,31,Q5L2C2,9897,1,0.317268
2,49,69,P30930,10663,1,0.730571
3,49,62,P30930,8862,1,0.730571
4,89,155,P70265,1795,1,0.271705


In [ ]:
# scratch = Path("/home/stef/quest_data/hiec/scratch")
# subdirs = ['sprhea_v3_folded_pt_ns', 'sprhea_v3_folded_n_100']
# split_strategies = ['rcmcs', 'homology']
# new_cols = ['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding', 'am_smarts', 'reaction_center', 'y']

# for subdir in subdirs:
#     sup_path = scratch / subdir
#     for split_strategy in split_strategies:
#         if split_strategy == 'homology' and subdir == 'sprhea_v3_folded_n_100':
#             continue

#         path = sup_path / split_strategy / "3fold"

#         for p in path.glob("*.parquet"):
#             print(p)
#             df = pd.read_parquet(p)
#             print(df.columns)
#             df.columns = new_cols
#             print(df.columns)
#             df.to_parquet(p)

/home/stef/quest_data/hiec/scratch/sprhea_v3_folded_pt_ns/rcmcs/3fold/test.parquet
Index(['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding',
       'smarts', 'reaction_center', 'y'],
      dtype='object')
Index(['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding',
       'am_smarts', 'reaction_center', 'y'],
      dtype='object')
/home/stef/quest_data/hiec/scratch/sprhea_v3_folded_pt_ns/rcmcs/3fold/train_val_1.parquet
Index(['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding',
       'smarts', 'reaction_center', 'y'],
      dtype='object')
Index(['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding',
       'am_smarts', 'reaction_center', 'y'],
      dtype='object')
/home/stef/quest_data/hiec/scratch/sprhea_v3_folded_pt_ns/rcmcs/3fold/train_val_0.parquet
Index(['protein_idx', 'reaction_idx', 'pid', 'rid', 'protein_embedding',
       'smarts', 'reaction_center', 'y'],
      dtype='object')
Index(['protein_idx', 'reaction_idx', 'pid', '

In [3]:
y_pred = df['logits'] > 0.5
(y_pred == df['y']).mean()

0.6222222222222222

In [20]:
fn = "/home/stef/quest_data/hiec/scratch/sprhea_v3_folded_pt_ns/rcmcs/3fold/train_val_1.parquet"
staged_on_quest = pd.read_parquet(fn)

In [29]:
issues = {}
for i, row in staged_on_quest.iterrows():
    sma = row['smarts']
    lhs, rhs = [side.split('.') for side in sma.split('>>')]
    lhs_mols = [Chem.MolFromSmiles(s) for s in lhs]
    rhs_mols = [Chem.MolFromSmiles(s) for s in rhs]
    l_rc, r_rc = [list(elt) for elt in row['reaction_center']]
    for m, r in zip(lhs_mols + rhs_mols, l_rc + r_rc):
        if r.max() > m.GetNumAtoms():
            issues[row['reaction_idx']] = (sma, [elt.max() for elt in l_rc + r_rc], [elt.GetNumAtoms() for elt in lhs_mols + rhs_mols])

In [30]:
issues

{4236: ('[OH2:10].[O:1]([P:2](=[O:3])([OH:4])[O:5][P:6](=[O:7])([OH:8])[OH:9])[P:11]([O:12][CH2:15][CH:16]1[CH:17]([OH:20])[CH2:19][CH:21]([n:22]2[cH:23][n:25][c:27]3[c:24]2[n:26][c:28]([NH2:31])[n:30][c:29]3[O:32][CH3:33])[O:18]1)(=[O:13])[OH:14]>>[OH:1][P:2](=[O:3])([OH:4])[O:5][P:6](=[O:7])([OH:8])[OH:9].[OH:10][P:11]([O:12][CH2:15][CH:16]1[CH:17]([OH:20])[CH2:19][CH:21]([n:22]2[cH:23][n:25][c:27]3[c:24]2[n:26][c:28]([NH2:31])[n:30][c:29]3[O:32][CH3:33])[O:18]1)(=[O:13])[OH:14]',
  [np.int64(0), np.int64(22), np.int64(21), np.int64(2)],
  [1, 32, 9, 24]),
 1399: ('[CH2:49]1[N:50]([CH3:75])[c:62]2[c:56]([cH:61][cH:64][c:67]3[c:65]2[cH:68][c:70]2[c:71]([cH:69]3)[O:73][CH2:74][O:72]2)-[c:53]2[c:51]1[c:52]1[c:54]([cH:58][c:57]2[O:63][CH3:66])[O:59][CH2:60][O:55]1.[c:1]1([C:7]([NH2:8])=[O:9])[cH:2][n+:3]([CH:10]2[O:11][CH:13]([CH2:16][O:18][P:19](=[O:20])([OH:21])[O:22][P:23](=[O:24])([OH:25])[O:26][CH2:27][CH:28]3[O:29][CH:31]([n:34]4[cH:36][n:39][c:41]5[c:37]4[n:40][cH:45][n:47][c:46]5

In [2]:
with open("/home/stef/quest_data/hiec/data/sprhea/v3_folded_pt_ns.json", 'r') as f:
    sprhea = json.load(f)

adj, idx_sample, idx_feature = construct_sparse_adj_mat(Path("/home/stef/quest_data/hiec/data/sprhea/v3_folded_pt_ns.csv"))

Constructing v3_folded_pt_ns sparse adjacency matrix


In [25]:
entry = sprhea[idx_feature[440]]
entry

{'smarts': 'CCCCCCCCCCCCCCCCOCC(COP(=O)(O)OCC[N+](C)(C)C)OC(=O)CCC.O>>CCCC(=O)O.CCCCCCCCCCCCCCCCOCC(O)COP(=O)(O)OCC[N+](C)(C)C',
 'min_rules': ['rule0007', 'rule0006'],
 'rcs': [[[18, 31], [0]], [[3, 5], [19]]],
 'enzymes': [{'uniprot_id': 'Q8VDG7',
   'sequence': 'MGAGQSVCFPPISGPHHIGCTDVMEGHSLEGSLFRLFYPCQASEKCEQPLWIPRYEYSMGLADYLQYNKRWVGLLFNVGIGSCRLPVSWNGPFKAKESGYPLIILSHGLGGFRASYSAFCMELASRGFVVAAVEHRDQSAAATYFCKPTSQESSPAESLEEEWLPFRRIKEGEKEFHVRNPQVHQRVKECVRVLRILQDASAGKTVVNVFPGGLDLMTLKGSIDRNRVAVMGHSFGGATAVLALTQEVQFRCAIALDAWMFPLERDFYPKARGPVFFINVEKFQTVESVNLMKKICAQHEQSRIVTVLGAVHRSQTDFAFVTGNLIAKFFSSNSRGTLDPYESQEVMVRAILAFLQKHLDLKEDYDQWSSFIEGVGPSLIQGAPHYLSSL',
   'existence': 'Evidence at protein level',
   'reviewed': 'reviewed',
   'ec': '2.3.1.149; 3.1.1.47',
   'organism': 'Mus musculus (Mouse)',
   'name': 'Platelet-activating factor acetylhydrolase 2, cytoplasmic (EC 3.1.1.47) (PAF:lysophospholipid transacetylase) (PAF:sphingosine transacetylase) (Platelet-activating factor acetyltransfera

In [32]:
for side in entry['smarts'].split('>>'):
    for s in side.split('.'):
        m = Chem.MolFromSmiles(s)
        print(s, m.GetNumAtoms())

for side in entry['am_smarts'].split('>>'):
    for s in side.split('.'):
        m = Chem.MolFromSmiles(s)
        print(s, m.GetNumAtoms())

CCCCCCCCCCCCCCCCOCC(COP(=O)(O)OCC[N+](C)(C)C)OC(=O)CCC 37
O 1
CCCC(=O)O 6
CCCCCCCCCCCCCCCCOCC(O)COP(=O)(O)OCC[N+](C)(C)C 32
[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:15][CH2:17][CH2:19][CH2:23][CH2:24][CH2:25][CH2:26][CH2:27][CH2:28][CH2:29][CH2:30][CH2:31][CH3:32])([CH2:4][O:6][P:8](=[O:10])([OH:11])[O:12][CH2:14][CH2:16][N+:18]([CH3:20])([CH3:21])[CH3:22])[O:33][C:34](=[O:35])[CH2:36][CH2:37][CH3:38] 37
[OH2:2] 1
[CH:1]([OH:2])([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:15][CH2:17][CH2:19][CH2:23][CH2:24][CH2:25][CH2:26][CH2:27][CH2:28][CH2:29][CH2:30][CH2:31][CH3:32])[CH2:4][O:6][P:8](=[O:10])([OH:11])[O:12][CH2:14][CH2:16][N+:18]([CH3:20])([CH3:21])[CH3:22] 32
[OH:33][C:34](=[O:35])[CH2:36][CH2:37][CH3:38] 6


In [33]:
for idx in issues:
    smarts = sprhea[idx_feature[idx]]['smarts']
    lhs, rhs = [side.split('.') for side in smarts.split('>>')]
    mols = [Chem.MolFromSmiles(s) for s in lhs + rhs]
    for rc_max, mol in zip(issues[idx][1], mols):
        if rc_max > mol.GetNumAtoms():
            print(f"idx {idx} with smarts {smarts} has issue: {rc_max} > {mol.GetNumAtoms()} for {Chem.MolToSmiles(mol)}")

In [24]:
staged_on_quest.loc[staged_on_quest.reaction_idx == 440]

,protein_idx,reaction_idx,pid,rid,protein_embedding,smarts,reaction_center,y
948,558,440,Q8VDG7,9888,"[0.07856997, 0.24768348, 0.07567654, 0.0232599...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",1
949,559,440,P79106,9888,"[0.09574465, 0.23234762, 0.058149666, 0.032877...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",1
950,560,440,Q99487,9888,"[0.092274666, 0.23454094, 0.05176036, 0.043576...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",1
951,561,440,P83006,9888,"[0.10112702, 0.24632981, 0.06338388, 0.0236249...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",1
10903,9971,440,P0DUN3,9888,"[0.16980408, 0.30399475, 0.009269477, -0.06162...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",0
...,...,...,...,...,...,...,...,...
93470,6924,440,A4TSJ5,9888,"[0.010831217, 0.090564646, 0.0048261955, 0.137...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",0
95428,333,440,Q9ERS7,9888,"[-0.052899245, 0.29523712, -0.069401555, 0.042...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",0
96850,3589,440,P27038,9888,"[0.07959237, 0.18772636, 0.08321738, -0.025589...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",0
97345,18501,440,Q9N0A4,9888,"[0.032802057, 0.27320108, 0.026131785, 0.06960...",[CH:1]([CH2:3][O:5][CH2:7][CH2:9][CH2:13][CH2:...,"[[[18, 31], [0]], [[3, 5], [19]]]",0


In [13]:
duplicate_am = {}
elt_mismatch = {}
for k, v in sprhea.items():
    sma = v['am_smarts']
    lhs, rhs = [side.split('.') for side in sma.split('>>')]
    lhs_mols = [Chem.MolFromSmiles(s) for s in lhs]
    rhs_mols = [Chem.MolFromSmiles(s) for s in rhs]
    l_ams = [a.GetAtomMapNum() for m in lhs_mols for a in m.GetAtoms()]
    r_ams = [a.GetAtomMapNum() for m in rhs_mols for a in m.GetAtoms()]
    l_idx_elt = {a.GetAtomMapNum(): a.GetAtomicNum() for m in lhs_mols for a in m.GetAtoms()}
    r_idx_elt = {a.GetAtomMapNum(): a.GetAtomicNum() for m in rhs_mols for a in m.GetAtoms()}
    
    if any([count > 1 for count in Counter(l_ams).values()]) or any([count > 1 for count in Counter(r_ams).values()]):
        duplicate_am[k] = v
        continue

    for am, elt in l_idx_elt.items():
        if r_idx_elt.get(am, elt) != elt:
            elt_mismatch[k] = v
            print(f"mismatch for am {am} with elt {elt} on lhs and {r_idx_elt.get(am, elt)} on rhs")
            break

    

    

    

In [14]:
len(duplicate_am), len(elt_mismatch)

(1, 0)

In [16]:
duplicate_am

{'14065': {'smarts': 'CC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)C1=CCC(C)=CC1.O>>CC1=CCC(C(C)CCC2C(C)(O)CCC3C2(C)CCC2C4(C)CCCC(C)(C)C4CCC23C)=CC1',
  'min_rules': ['rule0918', 'rule0620'],
  'rcs': [[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]],
   [[25, 26, 29, 21, 20, 32, 15, 16, 9, 10, 12]]],
  'enzymes': [{'uniprot_id': 'Q796C3',
    'sequence': 'MGTLQEKVRRFQKKTITELRDRQNADGSWTFCFEGPIMTNSFFILLLTSLDEGENEKELISSLAAGIHAKQQPDGTFINYPDETRGNLTATVQGYVGMLASGCFHRTEPHMKKAEQFIISHGGLRHVHFMTKWMLAANGLYPWPALYLPLSLMALPPTLPIHFYQFSSYARIHFAPMAVTLNQRFVLINRNISSLHHLDPHMTKNPFTWLRSDAFEERDLTSILLHWKRVFHAPFAFQQLGLQTAKTYMLDRIEKDGTLYSYASATIYMVYSLLSLGVSRYSPIIRRAITGIKSLVTKCNGIPYLENSTSTVWDTALISYALQKNGVTETDGSVTKAADFLLERQHTKIADWSVKNPNSVPGGWGFSNINTNNPDCDDTTAVLKAIPRNHSPAAWERGVSWLLSMQNNDGGFSAFEKNVNHPLIRLLPLESAEDAAVDPSTADLTGRVLHFLGEKVGFTEKHQHIQRAVKWLFEHQEQNGSWYGRWGVCYIYGTWAALTGMHACGVDRKHPGIQKALRWLKSIQNDDGSWGESCKSAEIKTYVPLHRGTIVQTAWALDALLTYENSEHPSVVKGMQYLTDSSSHSADSLAYPAGIGLPKQFYIRYHSYPYVFSLLAVGKYLDSIEKETANET',
    'existe

In [8]:
eg = '[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7]([CH3:8])=[CH:9][CH2:10][CH2:11][C:12]([CH3:13])=[CH:14][CH2:15][CH2:16][C:17]([CH3:18])=[CH:19][CH2:20][CH2:21][C:22]([CH3:23])=[CH:24][CH2:25][CH2:26][CH:27]([CH3:28])[C:29]1=[CH:30][CH2:31][C:32]([CH3:33])=[CH:34][CH2:35]1.[OH2:1]'
lhs = eg.split('.')
lhs_mols = [Chem.MolFromSmiles(s) for s in lhs]
l_idx_am = {a.GetIdx(): a.GetAtomMapNum() for m in lhs_mols for a in m.GetAtoms()}
l_idx_elt = {a.GetAtomMapNum(): a.GetAtomicNum() for m in lhs_mols for a in m.GetAtoms()}
    
if len(l_idx_am) != len(set(l_idx_am.values())):
    print("duplicate am")

In [10]:
l_idx_am

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35}

In [18]:
replacement= '[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13][C:6]([CH3:18])=[CH:7][CH2:16][CH2:17][C:10]([CH3:36])=[CH:1][CH2:23][CH2:21][C:4]([CH3:22])=[CH:5][CH2:19][CH2:20][C:8]([CH3:24])=[CH:9][CH2:25][CH2:26][CH:27]([CH3:28])[C:29]1=[CH:30][CH2:32][C:34]([CH3:35])=[CH:33][CH2:31]1.[OH2:11]>>[CH3:35][C:34]1=[CH:33][CH2:31][C:29]([CH:27]([CH3:28])[CH2:26][CH2:25][CH:9]2[C:10]([CH3:36])([OH:11])[CH2:17][CH2:16][CH:7]3[C:8]2([CH3:24])[CH2:20][CH2:19][CH:5]2[C:4]4([CH3:22])[CH2:21][CH2:23][CH2:1][C:2]([CH3:14])([CH3:15])[CH:3]4[CH2:12][CH2:13][C:6]23[CH3:18])=[CH:30][CH2:32]1'
sprhea['14065']['am_smarts'] = replacement

with open("/home/stef/quest_data/hiec/data/sprhea/v3_folded_pt_ns.json", 'w') as f:
    json.dump(sprhea, f)

In [23]:
staged_on_quest.loc[staged_on_quest.rid == '14065']

,protein_idx,reaction_idx,pid,rid,protein_embedding,smarts,reaction_center,y
5493,9808,4191,Q796C3,14065,"[-0.0789144, 0.08742644, 0.13630348, 0.1229612...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",1
13248,19766,4191,Q4PSY4,14065,"[0.0047013774, 0.19063234, -0.057207488, -0.01...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13249,11611,4191,P42593,14065,"[-0.026001442, 0.21024588, -0.1706128, -0.0033...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13250,11798,4191,O62698,14065,"[-0.108691245, 0.16667387, 0.20339926, 0.11039...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13251,16957,4191,Q9H5Q4,14065,"[-0.027651738, 0.18784207, -0.031290013, 0.034...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
...,...,...,...,...,...,...,...,...
62037,5200,4191,F1MAB7,14065,"[-0.09165915, 0.16170949, 0.14808396, 0.096084...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
62365,6219,4191,P0DTQ2,14065,"[0.09438254, 0.12517782, -0.006810354, 0.01648...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
62597,1280,4191,P34208,14065,"[0.0048641493, 0.20994161, -0.056773085, -0.00...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
63333,9485,4191,J7LQ09,14065,"[-0.18449304, 0.18624695, 0.08429514, -0.04473...",[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][CH2:6][C:7...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0


In [24]:
staged_on_quest.loc[staged_on_quest.rid == '14065', 'smarts'] = replacement
staged_on_quest.loc[staged_on_quest.rid == '14065']

,protein_idx,reaction_idx,pid,rid,protein_embedding,smarts,reaction_center,y
5493,9808,4191,Q796C3,14065,"[-0.0789144, 0.08742644, 0.13630348, 0.1229612...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",1
13248,19766,4191,Q4PSY4,14065,"[0.0047013774, 0.19063234, -0.057207488, -0.01...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13249,11611,4191,P42593,14065,"[-0.026001442, 0.21024588, -0.1706128, -0.0033...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13250,11798,4191,O62698,14065,"[-0.108691245, 0.16667387, 0.20339926, 0.11039...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
13251,16957,4191,Q9H5Q4,14065,"[-0.027651738, 0.18784207, -0.031290013, 0.034...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
...,...,...,...,...,...,...,...,...
62037,5200,4191,F1MAB7,14065,"[-0.09165915, 0.16170949, 0.14808396, 0.096084...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
62365,6219,4191,P0DTQ2,14065,"[0.09438254, 0.12517782, -0.006810354, 0.01648...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
62597,1280,4191,P34208,14065,"[0.0048641493, 0.20994161, -0.056773085, -0.00...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0
63333,9485,4191,J7LQ09,14065,"[-0.18449304, 0.18624695, 0.08429514, -0.04473...",[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13]...,"[[[1, 3, 6, 8, 11, 13, 16, 18, 21, 23], [0]], ...",0


In [25]:
staged_on_quest.to_parquet(fn, index=False)

In [ ]:
replacement= '[CH3:14][C:2]([CH3:15])=[CH:3][CH2:12][CH2:13][C:6]([CH3:18])=[CH:7][CH2:16][CH2:17][C:10]([CH3:36])=[CH:1][CH2:23][CH2:21][C:4]([CH3:22])=[CH:5][CH2:19][CH2:20][C:8]([CH3:24])=[CH:9][CH2:25][CH2:26][CH:27]([CH3:28])[C:29]1=[CH:30][CH2:32][C:34]([CH3:35])=[CH:33][CH2:31]1.[OH2:11]>>[CH3:35][C:34]1=[CH:33][CH2:31][C:29]([CH:27]([CH3:28])[CH2:26][CH2:25][CH:9]2[C:10]([CH3:36])([OH:11])[CH2:17][CH2:16][CH:7]3[C:8]2([CH3:24])[CH2:20][CH2:19][CH:5]2[C:4]4([CH3:22])[CH2:21][CH2:23][CH2:1][C:2]([CH3:14])([CH3:15])[CH:3]4[CH2:12][CH2:13][C:6]23[CH3:18])=[CH:30][CH2:32]1'
fn = "/home/stef/quest_data/hiec/scratch/sprhea_v3_folded_pt_ns/rcmcs/3fold/test.parquet"
hom = pd.read_parquet(fn)
hom.loc[hom.rid == '14065']

,protein_idx,reaction_idx,pid,rid,protein_embedding,smarts,reaction_center,y


: 

In [10]:
hom.loc[hom.rid == '14065', 'smarts'] = replacement
hom.to_parquet(fn, index=False)

In [1]:
from drfp import DrfpEncoder
from src.clip import EnzymeReactionCLIP

rxn_smiles = [
    "CO.O[C@@H]1CCNC1.[C-]#[N+]CC(=O)OC>>[C-]#[N+]CC(=O)N1CC[C@@H](O)C1",
    "CCOC(=O)C(CC)c1cccnc1.Cl.O>>CCC(C(=O)O)c1cccnc1",
]

fps = DrfpEncoder.encode(rxn_smiles)

In [2]:
fps

[array([0, 0, 0, ..., 0, 0, 0], shape=(2048,), dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], shape=(2048,), dtype=uint8)]

In [11]:
am_rxn = "[C:1][C:2]>>[C:1]=[C:2]"
rxn = "CC>>C=C"

am_fp = DrfpEncoder.encode(am_rxn)
fp = DrfpEncoder.encode(rxn)
(am_fp[0] == fp[0]).all()

np.False_

In [1]:
foo = [1,2,3]
max(foo)

3